In [3]:
import rasterio
from rasterio.mask import mask
from shapely.geometry import Polygon, mapping
import numpy as np

tiff_file = '/home/kea-trainee-punpun/Week2/data/CPM_lstm_2023_2.tiff'

polygon_coords = [
    (101.5, 15.5),
    (102.0, 15.5),
    (102.0, 16.0),
    (101.5, 16.0),   
    (101.5, 15.5)
]

#add
def get_tiff_bounds(tiff_file):
    with rasterio.open(tiff_file) as src:
        bounds = src.bounds
        crs = src.crs.to_string()
        return bounds, crs
#add
def convert_coordinates(coords, from_crs, to_crs):
    transformer = transform.Transformer.from_crs(from_crs, to_crs, always_xy=True)
    converted_coords = [transformer.transform(x, y) for x, y in coords]
    return converted_coords

def crop_raster_with_polygon(tiff_file, polygon_coords):
    # Open the TIFF file
    with rasterio.open(tiff_file) as src:
        # Create a Shapely polygon from the coordinates
        polygon = Polygon(polygon_coords)

        geojson_polygon = mapping(polygon)
        
        # Perform the rasterio mask
        out_image, out_transform = mask(src, [polygon], crop=True)
        
        return out_image, out_transform
#add
def is_polygon_within_bounds(polygon_coords, tiff_bounds):
    tiff_minx, tiff_miny, tiff_maxx, tiff_maxy = tiff_bounds
    polygon = Polygon(polygon_coords)
    polygon_bounds = polygon.bounds
    
    if (polygon_bounds[0] >= tiff_minx and polygon_bounds[1] >= tiff_miny and
        polygon_bounds[2] <= tiff_maxx and polygon_bounds[3] <= tiff_maxy):
        return True
    else:
        return False


In [4]:
crop_data, out_transform =crop_raster_with_polygon(tiff_file, polygon_coords)

rice_pixels = np.count_nonzero(crop_data == 1.000)
sugarcane_pixels = np.count_nonzero(crop_data == 2.000)

pixel_width = out_transform[0] 
pixel_height = -out_transform[4]

rice_area_sqm = rice_pixels * pixel_width * pixel_height
sugarcane_area_sqm = sugarcane_pixels * pixel_width * pixel_height

print(rice_area_sqm)
print(sugarcane_area_sqm)

0.06322456980348788
0.023409904839359206


In [9]:
def get_plant_type_by_polygon():
    
    data = [
    (101.5, 15.5),
    (102.0, 15.5),
    (102.0, 16.0),
    (101.5, 16.0),   
    (101.5, 15.5)
]
    # Validate input
    if 'polygon' not in data:
        return jsonify({'error': 'Polygon data is required'}), 400
    
    polygon_coords = data['polygon']
    
    # Check if polygon is within TIFF bounds
    tiff_bounds, tiff_crs = get_tiff_bounds(TIFF_FILE_PATH)
    
    # Convert polygon coordinates if necessary
    if tiff_crs != 'EPSG:4326':  # Assuming GeoTIFF CRS is EPSG:4326 (WGS84)
        polygon_coords = convert_coordinates(polygon_coords, 'EPSG:4326', tiff_crs)
    
    # Check if polygon is within bounds
    if not is_polygon_within_bounds(polygon_coords, tiff_bounds):
        return jsonify({'error': 'Polygon extends beyond the bounds of the GeoTIFF file'}), 400
    
    try:
        crop_data, out_transform = crop_raster_with_polygon(TIFF_FILE_PATH, polygon_coords)

        rice_pixels = np.count_nonzero(crop_data == 1.000)
        sugarcane_pixels = np.count_nonzero(crop_data == 2.000)

        pixel_width = out_transform[0]
        pixel_height = -out_transform[4]

        rice_area_sqm = rice_pixels * pixel_width * pixel_height
        sugarcane_area_sqm = sugarcane_pixels * pixel_width * pixel_height

        return jsonify({
            'polygon': polygon_coords,
            'plant_areas': {
                'rice_area_sqm': rice_area_sqm,
                'sugarcane_area_sqm': sugarcane_area_sqm
            }
        })
    except Exception as e:
        return jsonify({'error': str(e)}), 500

In [10]:
get_plant_type_by_polygon()

NameError: name 'jsonify' is not defined